<a href="https://colab.research.google.com/github/abduqodir-pulatov/3_module_imtihon/blob/main/3_module_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kagglehub gradio joblib

In [2]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import joblib
import gradio as gr
import matplotlib.pyplot as plt
import kagglehub

In [3]:
path = kagglehub.dataset_download("alessiocorrado99/animals10")
print("Path to dataset files:", path)

100%|██████████| 586M/586M [00:28<00:00, 21.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alessiocorrado99/animals10/versions/2


In [4]:
dataset_path = os.path.join(path, "raw-img")

In [5]:
categories = os.listdir(dataset_path)
print("Kategoriyalar:", categories)

images = []
labels = []

for category in categories:
    category_path = os.path.join(dataset_path, category)
    if os.path.isdir(category_path):
        for filename in os.listdir(category_path):
            if filename.lower().endswith(".jpg"):
                img_path = os.path.join(category_path, filename)
                try:
                    img = Image.open(img_path)
                    images.append(img)
                    labels.append(category)
                except Exception as e:
                    print(f"Xatolik {img_path} da: {e}")

print(f"Yuklangan rasmlar soni: {len(images)} | Label soni: {len(labels)}")


Kategoriyalar: ['ragno', 'gatto', 'gallina', 'scoiattolo', 'farfalla', 'mucca', 'pecora', 'cane', 'elefante', 'cavallo']
Yuklangan rasmlar soni: 1919 | Label soni: 1919


In [6]:
def convert_bytes_to_numbers(image):
    img = image.convert('L')  # Grayscale
    img = img.resize((64, 64))  # Kattaroq rasmlar juda og‘ir, 64x64 yetarli
    img = np.asarray(img)
    img = img.flatten()
    return img

In [7]:

X = np.array([convert_bytes_to_numbers(x) for x in images])
Y = np.array(labels)

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
svm = SVC(C=1.0, kernel='linear')
svm.fit(X_train, Y_train)

SVC(kernel='linear')

In [10]:
y_pred = svm.predict(X_test)
acc = accuracy_score(Y_test, y_pred)
print("Model aniqligi:", acc)

Model aniqligi: 0.2916666666666667


In [11]:
joblib.dump(svm, "svm_model.pkl")
print("Model saqlandi: svm_model.pkl")

Model saqlandi: svm_model.pkl


In [12]:
test_image1 = X_test[8]
result = svm.predict([test_image1])[0]
print(f"Model taxmini: {result}")

Model taxmini: gatto


In [13]:
model = joblib.load("svm_model.pkl")
label_names = sorted(list(set(Y)))  # noyob label nomlari

def predict_image(img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    img = img.convert('L').resize((64, 64))
    img_flat = np.array(img).flatten().reshape(1, -1)
    pred = model.predict(img_flat)[0]
    return f"Model taxmini: {pred}"

demo = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="numpy", label="Rasm yuklang"),
    outputs=gr.Textbox(label="Natija"),
    title="🐾 Hayvon rasmlarini SVM bilan aniqlash",
    description=f"SVM modeli yordamida hayvon rasmlarini klassifikatsiya qilish.\nModel aniqligi: {acc:.2f}"
)

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d51383b608cb80c878.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
